In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
dataset_ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")

In [4]:
len(dataset_ftse_mib)

1268

In [5]:
def get_stocks_by_week(df, week):
    """
    Returns a dataframe of stocks for a given week
    """
    return df[df['Week Number'] == week]

In [6]:
def get_portfolio_weekly_return(portfolio):
  totalReturn = 0
  
  for index, row in portfolio.iterrows():
    log_ret = row["Stock Weekly Return"]
    totalReturn+=log_ret
  return totalReturn/len(portfolio)


In [7]:
def get_ftse_mib_weekly_return(week_number):
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  prev_week_ret = ftse_mib.iloc[180+week_number-7]["log_ret"]
  next_week_ret = ftse_mib.iloc[180+week_number]["log_ret"]
  return next_week_ret-prev_week_ret


In [8]:
test = pd.read_csv("./results.csv")

In [9]:
test.head()

,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Risk,Res Std. Error,Stock Weekly Return
0,0,A2A.MI,0,-0.000713,0.779469,0.293579,0.000174,0.007567,-0.018795
1,1,A2A.MI,7,-0.000081,-0.035455,0.000753,0.000279,0.000026,0.003708
2,2,A2A.MI,14,-0.000423,-0.039381,0.000963,0.000132,0.000034,0.019859
3,3,A2A.MI,21,-0.000870,-0.048152,0.001466,0.000078,0.000052,-0.027421
4,4,A2A.MI,28,-0.001095,-0.067737,0.003573,0.000200,0.000146,0.020515


In [10]:
def get_sorted_df(dataframe, sort_method):
  dataframe = dataframe.sort_values(by=sort_method, ascending=False)
  return dataframe


In [11]:
def get_portfolio_risk(portfolio):
  risk = 0
  for index, row in portfolio.iterrows():
    risk += row["Risk"]/len(portfolio)
  return risk

Test: take top 15 and bottom 15 based on beta

In [12]:
final_dataframe = pd.DataFrame(columns=[
                               "Week", "portfolio Weekly Returns Beta", "portfolio Weekly Returns R-Squared", "portfolio Weekly Returns Risk", "portfolio Weekly Returns Res Std Error", "portfolio Weekly Returns Returns", "Ftse Mib weekly returns","Beta Risk","R-Squared Risk","Variance risk", "Residual-Variance Risk", "Return Risk"])
results = pd.read_csv("./results.csv")
for weekNumber in range(0,len(dataset_ftse_mib),7):
  res = get_stocks_by_week(test, weekNumber)
  res = res[res["Stock Symbol"] != "FTSEMIB.MI"]

  sorted_df_beta = get_sorted_df(res, "Beta")
  sorted_df_rsquared = get_sorted_df(res, "R-Squared")
  sorted_df_risk = get_sorted_df(res, "Risk")
  sorted_df_res_std_error = get_sorted_df(res, "Res Std. Error")
  sorted_df_ret = get_sorted_df(res, "Stock Weekly Return")

  frames_beta = [sorted_df_beta.head(4),  sorted_df_beta.tail(4)]
  frames_rsquared = [sorted_df_rsquared.head(4),  sorted_df_rsquared.tail(4)]
  frames_risk = [sorted_df_risk.head(4),  sorted_df_risk.tail(4)]
  frames_res_std_error = [sorted_df_res_std_error.head(
      4),  sorted_df_res_std_error.tail(4)]
  frames_ret = [sorted_df_ret.head(
      4),  sorted_df_ret.tail(4)]

  top_bottom_10_beta = pd.concat(frames_beta)
  top_bottom_10_rsquared = pd.concat(frames_rsquared)
  top_bottom_10_risk = pd.concat(frames_risk)
  top_bottom_10_res_std_error = pd.concat(frames_res_std_error)
  top_bottom_10_ret = pd.concat(frames_ret)

  risk_beta = get_portfolio_risk(top_bottom_10_beta)
  risk_rsquared= get_portfolio_risk(top_bottom_10_rsquared)
  risk_variance = get_portfolio_risk(top_bottom_10_risk)
  risk_errors = get_portfolio_risk(top_bottom_10_res_std_error)
  risk_returns = get_portfolio_risk(top_bottom_10_ret)
  if len(top_bottom_10_beta) > 0:
    portfolio_return_beta = get_portfolio_weekly_return(top_bottom_10_beta)
    portfolio_return_rsquared = get_portfolio_weekly_return(
        top_bottom_10_rsquared)
    portfolio_return_risk = get_portfolio_weekly_return(
        top_bottom_10_risk)
    portfolio_return_res_std_error = get_portfolio_weekly_return(
        top_bottom_10_rsquared)
    portfolio_return_ret = get_portfolio_weekly_return(
        top_bottom_10_ret)
    ftse_mib_return = get_ftse_mib_weekly_return(weekNumber)
    final_dataframe.loc[len(final_dataframe)] = [
        weekNumber, portfolio_return_beta, portfolio_return_rsquared,portfolio_return_risk,portfolio_return_res_std_error,portfolio_return_ret, ftse_mib_return, risk_beta, risk_rsquared, risk_variance, risk_errors, risk_returns]


In [13]:
final_dataframe.head()

,Week,portfolio Weekly Returns Beta,portfolio Weekly Returns R-Squared,portfolio Weekly Returns Risk,portfolio Weekly Returns Res Std Error,portfolio Weekly Returns Returns,Ftse Mib weekly returns,Beta Risk,R-Squared Risk,Variance risk,Residual-Variance Risk,Return Risk
0,0.0,-0.003105,-0.007558,0.001177,-0.007558,-0.005377,-0.008566,0.000253,0.000228,0.000315,0.000253,0.000283
1,7.0,-0.016110,-0.019398,-0.026967,-0.019398,-0.025536,-0.023240,0.000189,0.000125,0.000217,0.000128,0.000164
2,14.0,0.013424,0.008621,0.023690,0.008621,0.021041,0.020911,0.000441,0.000460,0.000647,0.000390,0.000483
3,21.0,-0.017608,-0.007148,-0.023863,-0.007148,-0.017509,-0.015410,0.000295,0.000161,0.000300,0.000275,0.000303
4,28.0,0.006544,0.006550,0.022502,0.006550,0.015595,0.015685,0.000263,0.000262,0.000501,0.000299,0.000463


In [14]:
import plotly.express as px


In [15]:
final_dataframe.head(1)

,Week,portfolio Weekly Returns Beta,portfolio Weekly Returns R-Squared,portfolio Weekly Returns Risk,portfolio Weekly Returns Res Std Error,portfolio Weekly Returns Returns,Ftse Mib weekly returns,Beta Risk,R-Squared Risk,Variance risk,Residual-Variance Risk,Return Risk
0,0.0,-0.003105,-0.007558,0.001177,-0.007558,-0.005377,-0.008566,0.000253,0.000228,0.000315,0.000253,0.000283


In [23]:
initial_investment = 100
portfolio_beta_total_return = initial_investment
portfolio_rsquared_total_return = initial_investment
portfolio_risk_total_return = initial_investment
portfolio_res_std_err_total_return = initial_investment
portfolio_returns_total_return = initial_investment
index_total_return = initial_investment
first = True
final_returns = pd.DataFrame(
    columns=["Week", "Beta Portfolio", "R-Squared Portfolio","Variance Portfolio","Residual variance Portfolio","Return Portfolio", "FTSE_MIB", "Beta Risk","R-Squared Risk", "Variance Risk","Error Risk","Returns risk"])
for index, row in final_dataframe.iterrows():
  
  portfolio_beta_total_return *= math.exp(row["portfolio Weekly Returns Beta"])
  portfolio_rsquared_total_return *= math.exp(
      row["portfolio Weekly Returns R-Squared"])
  portfolio_risk_total_return *= math.exp(
      row["portfolio Weekly Returns Risk"])
  portfolio_res_std_err_total_return *= math.exp(
      row["portfolio Weekly Returns Res Std Error"])
  portfolio_returns_total_return *= math.exp(
      row["portfolio Weekly Returns Returns"])
  index_total_return *= math.exp(row["Ftse Mib weekly returns"])
 
  final_returns.loc[len(final_returns)] = [row["Week"],
                                           portfolio_beta_total_return, 
                                           portfolio_rsquared_total_return, 
                                           portfolio_risk_total_return, 
                                           portfolio_res_std_err_total_return, 
                                           portfolio_returns_total_return, 
                                           index_total_return,
                                            row["Beta Risk"].mean()*len(final_dataframe), 
                                            row["R-Squared Risk"].mean()*len(final_dataframe), 
                                            row["Variance risk"].mean()*len(final_dataframe), 
                                            row["Residual-Variance Risk"].mean()*len(final_dataframe), 
                                            row["Return Risk"].mean()*len(final_dataframe)]
  

In [24]:
final_returns.head()

,Week,Beta Portfolio,R-Squared Portfolio,Variance Portfolio,Residual variance Portfolio,Return Portfolio,FTSE_MIB,Beta Risk,R-Squared Risk,Variance Risk,Error Risk,Returns risk
0,0.0,99.689944,99.247045,100.117728,99.247045,99.463785,99.147019,0.039535,0.035584,0.049117,0.039535,0.044106
1,7.0,98.096817,97.340389,97.453919,97.340389,96.956036,96.869363,0.029463,0.019441,0.033793,0.020020,0.025525
2,14.0,99.422513,98.183219,99.790190,98.183219,99.017723,98.916344,0.068775,0.071731,0.100905,0.060806,0.075310
3,21.0,97.687240,97.483937,97.437075,97.483937,97.299151,97.403750,0.046034,0.025171,0.046732,0.042877,0.047205
4,28.0,98.328559,98.124545,99.654431,98.124545,98.828436,98.943551,0.041075,0.040908,0.078088,0.046663,0.072293


In [19]:
len(final_returns)

156

## Calculation of portfolio risks and expected returns

### Portfolio risks

In [18]:
print("R-Squared portfolio risk: "+str(final_returns["Portfolio_Ret_R_Squared"].var()))
print("Beta portfolio risk: " +
      str(final_returns["Portfolio_Ret_Beta"].var()))
print("Risk portfolio risk: " +
      str(final_returns["Portfolio_Ret_Risk"].var()))
print("Res Std Error portfolio risk: " +
      str(final_returns["Portfolio_Ret_Std_Error"].var()))
print("Return portfolio risk: " +
      str(final_returns["Portfolio_Ret_Returns"].var()))
print("FTSE-MIB portfolio risk: "+str(final_returns["FTSE_MIB_Ret"].var()))


KeyError: 'Portfolio_Ret_R_Squared'

### Portfolios expected returns

In [ ]:
print("R-Squared portfolio expected return: " +
      str(final_returns["Portfolio_Ret_R_Squared"].mean()))
print("Beta portfolio expected return: " +
      str(final_returns["Portfolio_Ret_Beta"].mean()))
print("Risk portfolio expected return: " +
      str(final_returns["Portfolio_Ret_Risk"].mean()))
print("Res std error portfolio expected return: " +
      str(final_returns["Portfolio_Ret_Std_Error"].mean()))
print("Returns portfolio expected return: " +
      str(final_returns["Portfolio_Ret_Returns"].mean()))
print("FTSE-MIB portfolio expected return: " +
      str(final_returns["FTSE_MIB_Ret"].mean()))


R-Squared portfolio expected return: 99.1200181874493
Beta portfolio expected return: 102.0665938804765
Risk portfolio expected return: 102.48157611534738
Res std error portfolio expected return: 99.1200181874493
Returns portfolio expected return: 100.04348646913925
FTSE-MIB portfolio expected return: 99.2682898093567


In [ ]:
# first = True
# final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
# for index, row in final_dataframe.iterrows():
  
  
#   p_perc_return = (math.exp(row["portfolio Weekly Returns"]))
#   i_perc_return = (math.exp(row["Ftse Mib weekly returns"]))
#   if row["portfolio Weekly Returns"] < 0:
#     p_perc_return *=-1
#   if row["Ftse Mib weekly returns"]< 0:
#     i_perc_return *=-1
#   portfolio_total_return = initial_investment * \
#       p_perc_return if first else portfolio_total_return*p_perc_return
#   index_total_return = initial_investment * \
#       i_perc_return if first else index_total_return*i_perc_return
#   first = False
#   final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]

## Returns charts

In [ ]:
fig = px.line(final_returns, x="Week", y=["Beta Portfolio", "R-Squared Portfolio", "Variance Portfolio", "Residual variance Portfolio", "Return Portfolio", "FTSE_MIB"],
              title='Weekly returns, for an investment of 100€', labels=dict(value="Returns(€)", variable="Portfolio"), width=1100 )
fig.show()
